Para poder hacer el proceso de descarga hay que poner el entorno de ejecución al de TPUs, que nos dan casi 80GBs de memoria. Para ello seguimos los siguientes pasos:
Entorno de ejecución > Cambiar tipo de entorno de ejecución > Aceleración por hardware a TPU.


In [ ]:
from google.colab import drive  # Se conecta a drive
drive.mount('/content/drive')

# Imports

In [ ]:
from tensorflow.keras.applications import MobileNet
from tensorflow.keras.applications.mobilenet import preprocess_input
from tensorflow.keras.layers import Input, Dense, Flatten, Concatenate, LSTM, Bidirectional
from tensorflow.keras.models import Model
from tensorflow import keras
import tensorflow as tf

from skimage.io import imread

import pandas as pd
import numpy as np
import os

# Carga de datos (dataset AVA)

Primero se descargan las 255.000 imágenes del dataset AVA.

In [ ]:
# Descarga de Imagenes
!wget 'https://pruebasaluuclm-my.sharepoint.com/:u:/g/personal/fernando_rubio_uclm_es/ERGaDwGaYsVLg-5qpH4Sw_gBjet7G8WQAV2ijrMWu5bl9w?download=1'
!wget 'https://pruebasaluuclm-my.sharepoint.com/:u:/g/personal/fernando_rubio_uclm_es/EaNgOn3PuNdGlMx_qP6eshYBGYkcLuZtlrhphrdQcY_Ezg?download=1'
!wget 'https://pruebasaluuclm-my.sharepoint.com/:u:/g/personal/fernando_rubio_uclm_es/EdRxjrziO81HiCD9271rg9MBB4erjdKj3DFNvKUxcSYfbw?download=1'
!wget 'https://pruebasaluuclm-my.sharepoint.com/:u:/g/personal/fernando_rubio_uclm_es/EZntIFgpkyhFhfBGG5Qfz0oBvwbTMJZZEshbaVPoRtqKwg?download=1'

# Descarga de Ficheros
!wget 'https://pruebasaluuclm-my.sharepoint.com/:u:/g/personal/fernando_rubio_uclm_es/ESuuKua4j2hLvkFMxLNw84YBtTFAWu8uLxmXB--Pf-VwVQ?download=1'

Se extraen las imágenes y se guardan en la carpeta AVADataset.

In [ ]:
# Se extraen los archivos
# Imagenes
!mv 'ERGaDwGaYsVLg-5qpH4Sw_gBjet7G8WQAV2ijrMWu5bl9w?download=1' AVADataset.partaa
!mv 'EaNgOn3PuNdGlMx_qP6eshYBGYkcLuZtlrhphrdQcY_Ezg?download=1' AVADataset.partab
!mv 'EdRxjrziO81HiCD9271rg9MBB4erjdKj3DFNvKUxcSYfbw?download=1' AVADataset.partac
!mv 'EZntIFgpkyhFhfBGG5Qfz0oBvwbTMJZZEshbaVPoRtqKwg?download=1' AVADataset.partad
!cat AVADataset.part* > AVADataset.tar.gz
!rm -r AVADataset.part*
!tar -xzf AVADataset.tar.gz
!rm -r AVADataset.tar.gz

# Ficheros
!mv 'ESuuKua4j2hLvkFMxLNw84YBtTFAWu8uLxmXB--Pf-VwVQ?download=1' AVA_Dataset.zip
!unzip -q AVA_Dataset.zip

Se comprueba la cantidad de imágenes que hay en el directorio `AVADataset/`. En total debería de haber 255.353 (si se está trabajando con el dataset AVA completo.

In [ ]:
len(os.listdir("AVADataset/"))  # Se comprueba que se han descargado todas las imagenes: 255.353

Se descarga un archivo pandas con la información de las imágenes del dataset AVA.

In [ ]:
# Descarga del archivo con informacion de la bbdd
!wget 'https://pruebasaluuclm-my.sharepoint.com/:u:/g/personal/fernando_rubio_uclm_es/EZ8t-mcdR9tEjxPfzgmjbeIBDbs6PDnYbGsN2MOySInppA?download=1'
!mv 'EZ8t-mcdR9tEjxPfzgmjbeIBDbs6PDnYbGsN2MOySInppA?download=1' AVA_subset.zip
!unzip -q AVA_subset.zip
!wget 'https://pruebasaluuclm-my.sharepoint.com/:u:/g/personal/fernando_rubio_uclm_es/ESUYgue7Bv5DshQbI_QGYUoBwycoBSrw-0NkXzK-Qu71mA?download=1'
!mv 'ESUYgue7Bv5DshQbI_QGYUoBwycoBSrw-0NkXzK-Qu71mA?download=1' AVA_info.pklz

Se carga en un archivo pandas la bbdd. El `id` se corresponde con el nombre de las imágenes, y `Class` con el valor de la variable clase (0 = mala calidad, 1 = buena calidad).

In [ ]:
import gzip
import pickle
file_pickle = gzip.open('AVA_info.pklz','rb',2)
data = pd.read_pickle(file_pickle, compression=None)  # Se carga el dataframe

data

Se construye una bbdd solo con la columna id (contiene la direccion de los archivos) y la clase (0 o 1). Son las únicas columnas que se van a utilizar en este caso.

In [ ]:
data = data[['id', 'Class']]
data['id'] = data['id'].apply(lambda x: 'AVADataset/' + str(x) + '.jpg')
data['Class'] = data['Class'].apply(lambda x: str(x))

data

Se carga la X y la y a partir de la bbdd.

In [ ]:
# Se define la X (direccion de las imagenes)
X = data['id'].to_numpy()
# Se define y (0 1)
y = data['Class'].to_numpy()
y = y.astype(np.float)
y = tf.keras.utils.to_categorical(y, num_classes=2)

X.shape, y.shape

((255353,), (255353, 2))

# Preprocesamiento

Hay que preprocesar las imágenes. Para ello se extraen 9 parches y cada uno de estos parches se pasa por la red MobileNet, obteniendo 9 vectores de características de tamaño 1024.

Estos 9 vectores de características serán la entrada de los 3 modelos basados en mecanismos de atención y redes LSTM.

Se define el generador de parches. Recibe la dirección de la imagen a tratar junto con su variable clase y devuelve los 9 parches y la variable clase.

In [ ]:
class PatchGenerator(tf.keras.utils.Sequence):

    def __init__(self, files, labels, target_size = (224,224), 
                 batch_size = 32, nb_crops=3):
        self.files = files              # Direccion de las imagenes
        self.n_files = len(self.files)  # Cantidad de imagenes
        self.labels = labels            # Variable clase
        self.target_size = target_size  # Tamano de los parches
                                        # Por defecto (224x224)
        self.nb_channels = 3 # Numero de canales de las imagenes. 3 = RGB
        self.batch_size = batch_size
        self.nb_crops = nb_crops    # Cantidad de parches. 
        # Por defecto 3 (3 de ancho x 3 de alto = 9 parches totales)
        self.indexes = np.arange(self.n_files)

    def __len__(self):
        """
        Calcula el numero de batches por epoca
        """
        return int(np.ceil(self.n_files / self.batch_size))

    def __getitem__(self, index):
        """
        Genera un batch de datos
        """
        indexes = self.indexes[index*self.batch_size:min((index+1)*self.batch_size,self.n_files)]

        files_batch = self.files[indexes]
        labels_batch = self.labels[indexes]
        
        # Dimension de la variable X de un batch:
        # (tamano_batch, cantidad_total_parches, anchura_parches, altura_parches, numero_canales)
        # Por defecto --> (32, 9, 224, 224, 3)
        X = np.zeros((len(files_batch), self.nb_crops*self.nb_crops, self.target_size[0], self.target_size[1], self.nb_channels), dtype = np.float32)

        # Para cada imagen del batch
        for i in range(len(files_batch)):
          fn = files_batch[i]
          img = imread(fn)  # Se carga la imagen

          pad_x = 0
          pad_y = 0
          # Si la imagen es de menor tamano que target_size, se aplica padding para agrandarla
          if img.shape[0] <= self.target_size[0]:
              pad_y = (self.target_size[0] - img.shape[0]) // 2 + 1 
          if img.shape[1] <= self.target_size[1]:
              pad_x = (self.target_size[1] - img.shape[1]) // 2 + 1 
          # Si no es una imagen a color
          if len(img.shape) < 3:
              img = np.stack((img,)*3, axis=-1)
          img = np.pad(img, ((pad_y,pad_y), (pad_x,pad_x), (0,0)), mode='reflect') 

          # Guarda el ancho por alto de la imagen
          img_x, img_y = img.shape[1], img.shape[0] 
          # Formula para calcular la distancia entre parches: (distancia_x = tamano_original_x - tamano_parche_x) / x-1
          distancia_x = int((img_x - self.target_size[0]) / (self.nb_crops - 1))
          distancia_y = int((img_y - self.target_size[1]) / (self.nb_crops - 1))

          j=0
          for w in range(self.nb_crops):  # Para cada parche
            for z in range(self.nb_crops):
              # Se guardan las coordenadas de los parches
              crop = img[distancia_y*w:distancia_y*w + img_size, distancia_x*z:distancia_x*z + img_size, :].copy()
              X[i, j] = crop
              j=j+1

        return X, labels_batch

Se define el modelo que se encargará de preprocesar las imágenes.

In [ ]:
# Hiperparametros
nb_crops = 3
img_size = 224
nb_channels = 3

In [ ]:
# Dimensiones de entrada --> (9, 224, 224, 3)
image_shape = (nb_crops*nb_crops, img_size, img_size, nb_channels)
inp = tf.keras.layers.Input(shape=image_shape)

prep = tf.keras.applications.mobilenet.preprocess_input
core = tf.keras.applications.MobileNet(weights="imagenet", include_top=False, pooling="avg",
                                       input_shape=(img_size, img_size, nb_channels))
core.trainable = False  # Importante!!! MobileNet se marca como no entrenable

e = []
for i in range(nb_crops*nb_crops):   # Para cada parche creado
  # Se preprocesa; se pasa por mobilenet; se agrega una dimension
  e.append(tf.expand_dims(core(prep(inp[:, i])), 1))

out = tf.concat(e, axis=1)             # Se pegan los 9 parches

coder = Model(inputs=inp, outputs=out)

In [ ]:
coder.summary()

Hay que pasar todas las imágenes del dataset AVA por este modelo para preprocesarlas. Este es un proceso muy costoso y lleva mucho tiempo.

Como el servicio de Google Colab es limitado se ha hecho un bucle for y se han preprocesado las imágenes en grupos de 10.000. Cada uno de estos grupos ha tardado casi una hora en preprocesarse.

**Importante!!!** Modificar si se desea la dirección en la que se van a almacenar estos archivos:

In [ ]:
direccion_archivos = './drive/MyDrive/TFG/data/'

In [ ]:
for i in range(0, 26):
  x1 = X[i*10000:(i+1)*10000]
  y1 = y[i*10000:(i+1)*10000]

  # Se define el generador para los datos seleccionados
  generator = PatchGenerator(x1, y1)
  # Se preprocesan
  X_preprocesada = coder.predict_generator(generator)
  # Se almacenan los vectores de caracteristicas obtenidos
  # en un archivo numpy
  np.save(direccion_archivos+'X'+str(i+1), X_preprocesada)

## Construir dataset

A partir de los archivos guardados con el resultado del preprocesamiento, se van a guardar los vectores de características de cada imagen (cada imagen tiene 9 vectores, uno por parches). Se guardarán los vectores de cada imagen en un archivo `.npy` distinto.

**Importante!!!** Modificar si se desea la dirección en la que se van a almacenar las imágenes preprocesadas.

In [ ]:
direccion_datos = './drive/MyDrive/TFG/datos/'

In [ ]:
# Se crean 27 carpetas (una por cada 10.000 imagenes)
for i in range(1,27):
  os.mkdir(direccion_datos + 'X' + str(i))

In [ ]:
# Para cada carpeta se almacenan las 10.000 imagenes correspondientes
for i in range(0,27):
  X = np.load(direccion_archivos+'X'+str(i)+'.npy')
  for cont in range(0,len(X)):
    np.save(direccion_datos + 'X'+ str(i) +'/'+str(cont)+'.npy', X[cont])

  print("Iteracion ", i, ". Archivos: ", len(os.listdir(direccion_datos + 'X'+str(i))))

Todos estos archivos que están guardados en 27 carpetas se pueden comprimir manualmente en 5 archivos `.zip`. Estos archivos comprimidos se utilizarán en la libreta `Modelos.ipynb`, copiándose al entorno de ejecución y descomprimiéndolos.

Ahora se va a generar un `.csv` con la dirección que tendrá cada archivo dentro del directorio y la variable clase asociada.

In [ ]:
# Se genera un dataframe de pandas con la direccion de los archivos y la clase
for i in range(0,len(data)):
  # La columna id tendra un valor de la forma
  # --> Xi/numero.npy
  data['id'].loc[i] = 'X' + str(int(i/10000)+1) + '/' + str(i%10000) + '.npy'

data

In [ ]:
# Se guarda el .csv
data.to_csv(direccion_datos + 'dataset.csv')